In [1]:
import ast
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [8]:

# --- 4. SISTEMA DE RECOMENDACIÓN ---


df_peliculas_vectorizadas = pd.read_csv('./archivos/embedding_movies.csv')

# Usamos las variables globales definidas en la parte 1.
# La función ahora solo requiere el user_id.
def parse_embedding_string(embedding_str):
    """Convierte la cadena de texto de embedding a un array de NumPy."""
    try:
        # Usa ast.literal_eval para convertir la cadena de lista/array a una lista de Python
        return np.array(ast.literal_eval(embedding_str), dtype=np.float32)
    except (ValueError, TypeError):
        # Manejo de errores si la cadena está corrupta o vacía
        print(f"Advertencia: No se pudo parsear el embedding. Cadena: {embedding_str[:50]}...")
        # Devuelve un array de ceros del tamaño esperado (1536 para text-embedding-3-small)
        return np.zeros(1536, dtype=np.float32)

df_peliculas_vectorizadas['embedding'] = df_peliculas_vectorizadas['embedding'].apply(parse_embedding_string)

def recomendar_pelicula_a(user_id, top_n=5):

    
    # Referencia a las variables globales (los DataFrames vectorizados)
    # Reemplaza 'df_merged' y 'unique_movies_subset' con los nombres exactos de tus variables globales si son diferentes.
    df_merged=pd.read_csv('./archivos/dataset_merged.csv')
    df_interacciones = df_merged
    #df_peliculas_vectorizadas = pd.read_csv('./archivos/embedding_movies.csv')
    
    # PASO A: Filtrar qué le gusta al usuario (ej: Ratings >= 4)
    user_history = df_interacciones[
        (df_interacciones['userId'] == user_id) & 
        (df_interacciones['rating'] >= 4.0)
    ]
    
    if user_history.empty:
        return f"El usuario {user_id} no tiene suficientes ratings positivos (> 4.0) o no existe en el registro."
    
    liked_movie_ids = user_history['movieId'].values
    
    # PASO B: Recuperar los vectores de esas películas
    liked_vectors_df = df_peliculas_vectorizadas[df_peliculas_vectorizadas['movieId'].isin(liked_movie_ids)]
    
    if liked_vectors_df.empty:
        return f"El usuario {user_id} tiene gustos, pero ninguna de esas películas está en el catálogo vectorizado actual."

    # PASO C: Crear Vector de Usuario (Promedio)
    matrix_liked = np.stack(liked_vectors_df['embedding'].values)
    user_profile_vector = np.mean(matrix_liked, axis=0).reshape(1, -1)
    
    # PASO D: Calcular Similitud con TODO el catálogo disponible
    catalog_matrix = np.stack(df_peliculas_vectorizadas['embedding'].values)
    
    # Coseno entre [Perfil Usuario] vs [Catálogo]
    similarities = cosine_similarity(user_profile_vector, catalog_matrix)
    
    recommendations = df_peliculas_vectorizadas.copy()
    recommendations['similarity_score'] = similarities[0]
    
    # PASO E: Filtrar (Quitar las que ya vio) y Ordenar
    ids_vistos = df_interacciones[df_interacciones['userId'] == user_id]['movieId'].values
    
    recs_finales = recommendations[~recommendations['movieId'].isin(ids_vistos)]
    
    # Ordenar por similitud descendente y tomar Top N
    top_recs = recs_finales.sort_values(by='similarity_score', ascending=False).head(top_n)
    
    return top_recs[['title', 'vote_average', 'similarity_score','genres','overview']],user_history



In [11]:
df_merged=pd.read_csv('./archivos/dataset_merged.csv')
mi_usuario_a_probar = df_merged['userId'].iloc[15] # Tomando el primer usuario del DF

resultados,user_history = recomendar_pelicula_a(mi_usuario_a_probar)

print(f"\n🎬 Recomendaciones para Usuario {mi_usuario_a_probar}")
if isinstance(resultados, str):
    print(resultados)
else:
    print(resultados.to_markdown(index=False))


🎬 Recomendaciones para Usuario 2
| title               |   vote_average |   similarity_score | genres                | overview                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
|:--------------------|---------------:|-------------------:|:----------------------|:-----------------------------------------------------------------------

In [10]:
user_history

,userId,movieId,rating,title,overview,genres,vote_average
6,2,17,5.0,The Dark,Adèle and her daughter Sarah are traveling on ...,"['Horror', 'Thriller', 'Mystery']",5.6
8,2,110,4.0,Three Colors: Red,Red This is the third film from the trilogy by...,"['Drama', 'Mystery', 'Romance']",7.8
10,2,150,5.0,48 Hrs.,A hard-nosed cop reluctantly teams up with a w...,"['Thriller', 'Action', 'Comedy', 'Crime', 'Dra...",6.5
11,2,153,4.0,Lost in Translation,"Two lost souls visiting Tokyo -- the young, ne...",['Drama'],7.3
17,2,222,5.0,Berlin: Symphony of a Great City,Berlin is Walter Ruttmanns black and white doc...,"['Documentary', 'History']",8.0
22,2,253,4.0,Live and Let Die,James Bond must investigate a mysterious murde...,"['Adventure', 'Action', 'Thriller']",6.4
23,2,261,4.0,Cat on a Hot Tin Roof,"Brick, an alcoholic ex-football player, drinks...","['Drama', 'Romance']",7.6
24,2,265,5.0,Lili Marleen,Lili Marleen is the story of a German singer n...,"['War', 'Drama', 'Romance']",5.6
25,2,266,5.0,Contempt,Screenwriter Paul Javal's marriage to his wife...,['Drama'],7.5
27,2,273,4.0,The White Sound,"Lukas, a young schizophrenic man, has to deal ...",['Drama'],7.0
